We want to optimize the digital circuit depicted below, where each small box with a number represents a gate realizing
 a logical operation.

![title](img/circuit.png) 

Each gate has a nominal area $A_i$ and we must optimize some scaling factors $x_i\geq 1$, which affect
 the total area $A=\sum_i x_i A_i$ of the circuit, as well as its delay and power consumption. The
 power consumption is an affine function $P=P_0+\sum_i x_i P_i$ of the scaling factors, and we admit that
 the delay of gate $i$ can be modelled by
 $$D_i(\boldsymbol{x}) = \left\{\begin{array}{ll}
 \frac{\gamma_i}{x_i}\ C_i^{out} & \textrm{ if $i$ is an output gate}\\
  \frac{\gamma_i}{x_i}\sum_{j\in\delta^+(i)} (\alpha_j+\beta_j x_j) & \textrm{ otherwise.}
                     \end{array}
\right.
,$$
 where the $C_i^{out},\alpha_i,\beta_i,\gamma_i$'s are positive physical constants of the gates,
 and $\delta_+(i)$ is the set of successors of $i$. The delay of the circuit is the maximum
 delay over all paths from the input to the output layer, and the delay of a path is simply the sum of delays of the gates on this path.
 
The goal of this exercise is to formulate the problem of minimizing the delay of the circuit depicted above, subject to $A\leq A^{\rm max}, P\leq P^{\rm max},$ as a GP (in posynomial form), and to solve its convex log-sum-exp reformulation.

Let us first import the necessary packages and generate some data for this problem:

In [ ]:
import picos
import networkx as nx
import numpy as np

In [ ]:
#This is the circuit's graph. Nodes are indexed starting from 0. We add a dummy node called 'out' for the output.
G = nx.DiGraph()
G.add_edges_from([(0,3),(1,3),(1,4),(2,3),(2,4),(3,5),(3,6),(4,6),(5,'out'),(6,'out')])
n = G.number_of_nodes() - 1
#output gates
T=[5,6]


#Generate some constants 
#NB: some of them are not used, such as C_out[i] for non-output gates
np.random.seed(42)
alpha = np.random.rand(n)
beta = np.random.rand(n)
gamma = np.random.rand(n)
AA = np.random.rand(n)
PP = np.random.rand(n)
P0 = np.random.rand()
C_out = 5 * np.random.rand(n)

Now, we write a function that calculates the delay of the circuit

In [ ]:
def delay_gate(x,i):
    S = 0.
    for j in G.successors(i):
        if j=='out':
            S += C_out[i]
        else:
            S += alpha[j]+beta[j]*x[j]
    
    return gamma[i]/x[i] * S
        
def total_delay(x):
    for e1,e2 in G.edges():
        G[e1][e2]['weight'] = delay_gate(x,e1)
    return nx.dag_longest_path_length(G)

We generate a random solution $x_0 \geq 1$, and define $Amax$ and $Pmax$ accordingly

In [ ]:
x0 = np.exp(2 * np.random.rand(n))
Amax = x0.dot(AA)
Pmax = P0 + x0.dot(PP)


print 'x:\n',x0
print 'Area of this solution: Amax=',Amax
print 'Power consumption of this solution: Pmax=',Pmax
print 'Delay of this solution: D(x)=',total_delay(x0)

Let $\delta_i$ be a variable for the delay $D_i(x)$ at gate $i\in[n]$, and let $\zeta_i$ be a variable for the total delay between the input and the (entrance) of gate $i\in[n]$. In addition, let $\tau$ be a variable for the total delay of the circuit. In the cell below, we write the inequalities that the $\delta_i$'s, $\zeta_i$'s, and $\tau$ must satisfy (it is enough to give lower bounds on the $\delta_i's$, $\zeta_i$'s,and $\tau$, since we want to *minimize* these variables). 

For a non-output gate, this variable must satisfy the posynomial constraint
$$
\delta_i \geq  \frac{\gamma_i}{x_i}\sum_{j\in\delta^+(i)} (\alpha_j+\beta_j x_j)
$$
and for a output gate $i$ it holds
$$
\delta_i \geq  \frac{\gamma_i}{x_i}C_i^{out}.
$$

The variables $\zeta_i\geq 0$ clearly satisfy
$$
\zeta_i \geq \max_{j\in\delta^-(i)}\ \zeta_j + \delta _j,\quad \forall i\in[n].
$$

Finaly, the total delay is the maximum of the delays at output gates:
$$
\tau \geq \max_{j\in T}\ \zeta_j + \delta _j.
$$

<span style="color:blue">
Reformulate the above constraints in posynomial form, i.e., each constraint must be formulated as $f(x)\leq 1$ for some posynomial $f$.
</span>

Delay at a non-output gate:
$$
(...) \leq 1,\quad \forall i \notin T.
$$

Delay at output gate:
$$
(...) \leq 1,\quad \forall i \in T.
$$

Delay between input and node $i$:
$$
(...) \leq 1,\quad \forall i\in[n],\forall j \in \delta^-(i).
$$
Total delay:
$$
(...) \leq 1,\quad \forall j \in T.
$$

<span style="color:blue">    
Perform the changes of variable $\delta_i=e^{d_i}$, $\zeta_i=e^{z_i}$, $x_i=e^{y_i}$, and $\tau=e^t$, and rewrite the above inequalities in log-sum-exp form, as well as the area and power consumption constraints. 
</span>

Delay at a non-output gate:
$$
\log(...) \leq  0,\quad \forall i\notin T.
$$

Delay at output gate:
Since it is a monomial, after the change of variable we get a linear constraint:
$$
(...)
$$

Delay between input and node $i$:
$$
\log(...)\leq 0,\quad \forall j \in \delta^-(i).
$$
Total delay:
$$
\log(...)\leq 0,\quad \forall j \in T.
$$

After the change of variables, the power consumption constraint $P_0 + \sum_i x_i P_i\leq P_{max}$
becomes
$$
\log(...) \leq 0.
$$
Similarly, the area constraint is
$$
\log(...) \leq 0.
$$


<span style="color:blue">    
So, the problem of minimizing the delay can be formulated as minimizing $t$ subject to the above constraints, and the nonnegativity of the $y_i$'s (because x_i\geq 1). Formulate this problem using picos. 
</span>

<span style="color:blue">    
*Hint*: If the z_i's are some variables, the simplest to enter a log-sum-exp constraint of the form
    $log(\sum_{j=0}^{n-1} e^{z_i-z_j})\leq 0$ in picos (this is a dummy example) is to write
    ``P.add_constraint(picos.log(sum([picos.exp(z[i]-z[j]) for j in range(n)]))<=0)``. To take the logarithm of a
    **constant** (such as $\gamma_i$), you can write ``np.log(gamma[i])``.
    
    
</span>

In [ ]:
#non-output nodes:
NT = [i for i in range(n) if i not in T]

#create the problem
P = picos.Problem()

#define the decision variables
d = P.add_variable('d',n)
y = P.add_variable('y',n)
z = P.add_variable('z',n)
t = P.add_variable('t',1)

#add the constraints
#TODO(...)

#Define the objective function
#TODO (...)

#solve the problem
sol = P.solve()

<span style="color:blue">   
If you have implemented the exponential cone program correctly, executing the next cell will display statistics of the optimal solution. In particular, you should check that the delay of the optimal solution (as computed with the function ``total_delay()`` coincides with the exponential of the objective value of the problem).
</span>

In [ ]:
#retrieve scaling factor by taking the exponential of the optimal `y`
x = np.exp(y.value).ravel()

print 'x:\n',x
print 'Area of this solution: A=', AA.dot(x),'<=',Amax
print 'Power consumption of this solution: P=', P0 + PP.dot(x),'<=',Pmax
print 'Delay of this solution: D(x)=',total_delay(x)
print 'exp of objective value of the problem=',np.exp(P.obj_value())
print 'improvement of delay with respect to initial solution x_0:', 100 * (1-total_delay(x)/total_delay(x0)),'%'
